In [12]:
from pathlib import Path
import numpy as np
import pandas as pd
import hdbscan

from collections import Counter
from sklearn.cluster import KMeans, DBSCAN
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import silhouette_score

In [13]:
here = Path.cwd()
root = next(p for p in [here, *here.parents] if (p / "data" / "processed").exists())
df = pd.read_csv(root / "data" / "processed" / "state1" / "ios_df.csv", header=0)
df = df[df['eventType'] == 5]
df.shape

(3457, 12)

In [14]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df)
scaled_df = pd.DataFrame(X_scaled, columns=df.columns)

In [15]:
def calculate_gini_score(labels):
    counts = Counter(labels)
    if -1 in counts:
        del counts[-1]
    if len(counts) < 2:
        return 0.0 
    y = np.array(sorted(counts.values()))
    n = len(y)
    numerator = sum((2 * (i + 1) - n - 1) * y[i] for i in range(n))
    denominator = n * sum(y)

    gini = numerator / denominator
    
    return gini

In [16]:
# k means model
results_list = []
# possible k values
for k in [2, 3, 4, 5]:
    kmeans_final = KMeans(n_clusters=k, random_state=42)
    kmeans_final.fit(scaled_df)
    final_labels = kmeans_final.labels_

    sil_score = silhouette_score(scaled_df, final_labels)
    noise_percentage = 0.0
    gini_score = calculate_gini_score(final_labels)

    results_list.append({
        'model': 'kmeans',
        'sil. #': sil_score,
        'Gini': gini_score,
        'noise %': noise_percentage,
        'clust. #': k
    })    


In [17]:
N_total = X_scaled.shape[0]

In [18]:
# DBSCAN modeling
for eps in np.linspace(0.1, 2.0, 20):
    for min_pts in [2, 40]:
        db = DBSCAN(eps=eps, min_samples=min_pts, n_jobs=-1).fit(X_scaled)
        labels = db.labels_
        
        non_noise_mask = labels != -1
        data_non_noise = X_scaled[non_noise_mask]
        labels_non_noise = labels[non_noise_mask]
        n_clusters = len(np.unique(labels_non_noise))
        n_noise = np.sum(labels == -1)
    
        if n_clusters >= 2 and len(labels_non_noise) >= 2:
            sil_score = silhouette_score(data_non_noise, labels_non_noise)
        else:
            sil_score = np.nan 
            
        noise_percentage = (n_noise / N_total) * 100
        gini_score = calculate_gini_score(labels) 

        results_list.append({
            'model': 'dbscan',
            'HP (eps)': eps, 
            'HP (min_samp)': min_pts,
            'sil. #': sil_score,
            'Gini': gini_score,
            'noise %': noise_percentage,
            'clust. #': n_clusters
        })

In [19]:
# HDBSCAN modeling
for min_samp in [2, 15]:
    clusterer = hdbscan.HDBSCAN(min_samples=min_samp)
    clusterer.fit(X_scaled)
    labels = clusterer.labels_
    
    non_noise_mask = labels != -1
    data_non_noise = X_scaled[non_noise_mask]
    labels_non_noise = labels[non_noise_mask]
    n_clusters = len(np.unique(labels_non_noise))
    n_noise = np.sum(labels == -1)

    if n_clusters >= 2 and len(labels_non_noise) >= 2:
        sil_score = silhouette_score(data_non_noise, labels_non_noise)
    else:
        sil_score = np.nan 
        
    noise_percentage = (n_noise / N_total) * 100
    gini_score = calculate_gini_score(labels) 

    results_list.append({
        'model': 'hdbscan',
        'HP (min_samp)': min_samp,
        'sil. #': sil_score,
        'Gini': gini_score,
        'noise %': noise_percentage,
        'clust. #': n_clusters
    })

/Users/shazi/Documents/VS Code/Arity - BTT Project 1/ARITY-BTT-PROJECT-1/.venv/lib/python3.13/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Users/shazi/Documents/VS Code/Arity - BTT Project 1/ARITY-BTT-PROJECT-1/.venv/lib/python3.13/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Users/shazi/Documents/VS Code/Arity - BTT Project 1/ARITY-BTT-PROJECT-1/.venv/lib/python3.13/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Users/shazi/Documents/VS Code/Arity - BTT Project 1/ARITY-BTT-PROJECT-1/.venv/lib/python3.13/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' 

In [20]:
results_df = pd.DataFrame(results_list)
results_df.head

<bound method NDFrame.head of       model    sil. #      Gini     noise %  clust. #  HP (eps)  HP (min_samp)
0    kmeans  0.304435  0.212467    0.000000         2       NaN            NaN
1    kmeans  0.151287  0.122071    0.000000         3       NaN            NaN
2    kmeans  0.174873  0.304093    0.000000         4       NaN            NaN
3    kmeans  0.178644  0.317501    0.000000         5       NaN            NaN
4    dbscan       NaN  0.000000   99.942146         1       0.1            2.0
5    dbscan       NaN  0.000000  100.000000         0       0.1           40.0
6    dbscan  0.887521  0.000000   99.421464        10       0.2            2.0
7    dbscan       NaN  0.000000  100.000000         0       0.2           40.0
8    dbscan  0.506990  0.145724   94.474978        78       0.3            2.0
9    dbscan       NaN  0.000000  100.000000         0       0.3           40.0
10   dbscan  0.233871  0.429435   83.627423       149       0.4            2.0
11   dbscan       NaN 

In [21]:
results_df = pd.DataFrame(results_list).sort_values(by=['clust. #', 'Gini', 'noise %', 'sil. #'], ascending=[False, True,True, False])
filtered_results = results_df[
    (results_df['noise %'] <= 20) &
    (results_df['clust. #'] <= 8)
]
print(filtered_results.head(20))

      model    sil. #      Gini    noise %  clust. #  HP (eps)  HP (min_samp)
42   dbscan  0.253979  0.869126   1.504194         8       2.0            2.0
3    kmeans  0.178644  0.317501   0.000000         5       NaN            NaN
2    kmeans  0.174873  0.304093   0.000000         4       NaN            NaN
1    kmeans  0.151287  0.122071   0.000000         3       NaN            NaN
44  hdbscan  0.549620  0.660387   1.735609         3       NaN            2.0
0    kmeans  0.304435  0.212467   0.000000         2       NaN            NaN
43   dbscan       NaN  0.000000   7.405265         1       2.0           40.0
41   dbscan       NaN  0.000000   8.822679         1       1.9           40.0
39   dbscan       NaN  0.000000  10.673995         1       1.8           40.0
37   dbscan       NaN  0.000000  12.641018         1       1.7           40.0
35   dbscan       NaN  0.000000  15.099798         1       1.6           40.0
33   dbscan       NaN  0.000000  18.686723         1       1.5  